In [1]:
from Datos import Datos
from EstrategiaParticionado import *
from Clasificador import *
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

<h3>Prueba de la clase EstrategiaParticionado<h3>

In [2]:
k = 3
p = 0.3
nEjecuciones = 3

n_datos = 10

dataset = Datos("tic-tac-toe.data")

#Cambio estrategia particionado comentar

Se prueba primero ValidacionSimple. Para visualizar mejor la partición, solo se usan los n_datos = 10 primeros datos del dataset. En general, el parametro n_datos debería ser igual al número de datos en el dataset. 

In [3]:

estrategia_simple = ValidacionSimple(p, nEjecuciones)
particiones_simple = estrategia_simple.creaParticiones(n_datos)

for particion in particiones_simple:
    datos_test = dataset.extraeDatos(particion.indicesTest)
    datos_train = dataset.extraeDatos(particion.indicesTrain)
    print("Test:", str(datos_test))
    print("Train:", str(datos_train))
    print()
    

Test: [array(['x', 'x', 'x', 'x', 'o', 'o', 'o', 'o', 'x', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'o', 'o', 'x', 'o', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'o', 'b', 'o', 'b', 'positive'],
      dtype=object)]
Train: [array(['x', 'x', 'x', 'x', 'o', 'b', 'o', 'o', 'b', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'o', 'x', 'o', 'o', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'b', 'o', 'o', 'o', 'b', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'o', 'b', 'b', 'o', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'o', 'o', 'b', 'b', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'b', 'b', 'o', 'o', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'b', 'o', 'b', 'o', 'positive'],
      dtype=object)]

Test: [array(['x', 'x', 'x', 'x', 'o', 'b', 'o', 'b', 'o', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o',

Prueba de ValidacionCruzada con parametro k = 3. Igual que en el caso anterior, para facilitar la visualización solose usan los 10 primeros datos (n_datos). 

In [4]:
estrategia_cruzada = ValidacionCruzada(k)
particiones_cruzada = estrategia_cruzada.creaParticiones(n_datos)

for particion in particiones_cruzada:

    datos_test = dataset.extraeDatos(particion.indicesTest)
    datos_train = dataset.extraeDatos(particion.indicesTrain)
    print("Test:", str(datos_test))
    print("Train:", str(datos_train))
    print()

Test: [array(['x', 'x', 'x', 'x', 'o', 'o', 'o', 'x', 'o', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'b', 'b', 'o', 'o', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'o', 'b', 'b', 'o', 'positive'],
      dtype=object)]
Train: [array(['x', 'x', 'x', 'x', 'o', 'o', 'b', 'o', 'b', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'b', 'o', 'o', 'o', 'b', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'o', 'x', 'o', 'o', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'b', 'o', 'o', 'b', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'o', 'o', 'o', 'x', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'o', 'o', 'b', 'b', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'o', 'b', 'o', 'b', 'o', 'positive'],
      dtype=object)]

Test: [array(['x', 'x', 'x', 'x', 'o', 'o', 'b', 'o', 'b', 'positive'],
      dtype=object), array(['x', 'x', 'x', 'x', 'b',

<h3>Prueba de la clase Clasificacion<h3>

Primero se necesita obtener los datos en un formato correcto.

In [5]:
archivo = 'tic-tac-toe.data'

dataset = Datos(archivo)

datos = dataset.datos
# Utilizamos el diccionario para adaptar los datos
for i in range(datos.shape[0]):
    for j in range(datos.shape[1]):
        if dataset.nominalAtributos[j]:
            datos[i, j] = dataset.diccionario[j][datos[i, j]]  

Después, crear las particiones datostrain y datostest. En este caso de prueba utilizaremos validacionSimple con nEjecuciones = 1.

In [6]:
p = 0.1
vs = ValidacionSimple(p, 1)
particiones = vs.creaParticiones(len(dataset.datos), seed=0)

datostrain = dataset.extraeDatos(particiones[0].indicesTrain)
datostest = dataset.extraeDatos(particiones[0].indicesTest)

Ahora, se entrena el modelo utilizando el conjunto datostrain

In [7]:
cl = ClasificadorNaiveBayes()
cl.entrenamiento(datostrain, dataset.nominalAtributos, dataset.diccionario)

Y, por último se predice la clase para datostest utilizando los porcentajes obtenidos. 

In [8]:
prediccion = cl.clasifica(datostest, dataset.nominalAtributos, dataset.diccionario)

Mostramos por pantalla el resultado obtenido:

In [9]:
cont = 0
for i in range(len(datostest)):
    if datostest[i][-1] == prediccion[i]:
        cont += 1

print(cont, len(datostest), cont/len(datostest))

71 96 0.7395833333333334


In [10]:
print(1-cl.error(datostest, prediccion))

0.7395833333333333


Hemos obtenido aproximadamente un 74% de aciertos en este dataset y particion. Vamos a probar ahora los datos "german.data" haciendo las mismas operaciones que en el caso anterior.

In [11]:
archivo = 'german.data'

dataset = Datos(archivo)

datos = dataset.datos
for i in range(datos.shape[0]):
    for j in range(datos.shape[1]):
        if dataset.nominalAtributos[j] or j == datos.shape[1] - 1:
            datos[i, j] = dataset.diccionario[j][datos[i, j]]  


In [12]:
p = 0.1
vs = ValidacionSimple(p, 1)
particiones = vs.creaParticiones(len(dataset.datos), seed=0)

datostrain = dataset.extraeDatos(particiones[0].indicesTrain)
datostest = dataset.extraeDatos(particiones[0].indicesTest)

In [13]:
cl = ClasificadorNaiveBayes()
cl.entrenamiento(datostrain, dataset.nominalAtributos, dataset.diccionario)

In [14]:
prediccion = cl.clasifica(datostest, dataset.nominalAtributos, dataset.diccionario)

In [15]:
cont = 0
for i in range(len(datostest)):
    if datostest[i][-1] == prediccion[i]:
        cont += 1

print(cont, len(datostest), cont/len(datostest))

72 100 0.72


Igual que en el caso anterior, el modelo acierta en aproximadamente 70% de los casos. 

In [7]:
mnb = MultinomialNB(alpha=0)

datostrain = np.matrix(datostrain, dtype='int32')
datostest = np.matrix(datostest, dtype='int32')

X_train = datostrain[:, :-1]
y_train = datostrain[:, -1]

X_test = datostest[:, :-1]
y_test = datostest[:, -1]

"""
X = datos[:, :-1]
y = datos[:, -1]

X = X.astype('int32')
y = y.astype('int32')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=p, random_state=0)
"""
mnb.fit(X_train, y_train)

y_pred = mnb.predict(X_test)
from sklearn import metrics

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.65


c:\users\elena\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\users\elena\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [13]:
print(mnb.predict_proba(X_test))

[[0.33368026 0.66631974]
 [0.27029134 0.72970866]
 [0.30674323 0.69325677]
 [0.34378295 0.65621705]
 [0.44407758 0.55592242]
 [0.36553752 0.63446248]
 [0.33043334 0.66956666]
 [0.40669287 0.59330713]
 [0.33285795 0.66714205]
 [0.34602318 0.65397682]
 [0.19083471 0.80916529]
 [0.25427825 0.74572175]
 [0.34945605 0.65054395]
 [0.25372039 0.74627961]
 [0.35523539 0.64476461]
 [0.33386105 0.66613895]
 [0.33434544 0.66565456]
 [0.53147214 0.46852786]
 [0.48752297 0.51247703]
 [0.42537895 0.57462105]
 [0.37003041 0.62996959]
 [0.35542954 0.64457046]
 [0.32503743 0.67496257]
 [0.2797779  0.7202221 ]
 [0.26522096 0.73477904]
 [0.36936804 0.63063196]
 [0.31577122 0.68422878]
 [0.25041526 0.74958474]
 [0.38600337 0.61399663]
 [0.42311253 0.57688747]
 [0.36935964 0.63064036]
 [0.39337266 0.60662734]
 [0.32459051 0.67540949]
 [0.33501913 0.66498087]
 [0.3948114  0.6051886 ]
 [0.37397603 0.62602397]
 [0.34730217 0.65269783]
 [0.30295173 0.69704827]
 [0.33303433 0.66696567]
 [0.40035666 0.59964334]
